# 0.0 Imports

In [ ]:
import math
import pandas as pd
import numpy as np
import inflection
#import seaborn as sns

## 0.1. Helper Functions

## 0.2. Loading Data

In [ ]:
# o argumento low_memory fala para função pd.read para ler todo o arquivo de uma só vez
df_sales_row = pd.read_csv('data/train.csv', low_memory=False)
df_store_row = pd.read_csv('data/store.csv', low_memory=False)

#merge
df_raw = pd.merge(df_sales_row, df_store_row, how='left', on ='Store')

In [ ]:
df_raw.sample()

# 1.0 Descrição dos Dados

## 1.1. Rename Columns

In [ ]:
df1 = df_raw.copy()

In [ ]:
df1.columns

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo','StateHoliday', 'SchoolHoliday',
            'StoreType', 'Assortment','CompetitionDistance', 'CompetitionOpenSinceMonth','CompetitionOpenSinceYear', 
            'Promo2', 'Promo2SinceWeek','Promo2SinceYear', 'PromoInterval'] 

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

#Rename
df1.columns = cols_new

## 1.2. Data Dimension

In [ ]:
print('Number rows: {}'.format(df1.shape[0]))
print('Number cols: {}'.format(df1.shape[1]))

## 1.3. Data Types

In [ ]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

## 1.4. Check NA

In [ ]:
df1.isna().sum()

## 1.5. Fillout NA

In [ ]:
df1['competition_distance'].max()

In [ ]:
#competition_distance 
# Ela vai mostrar se na coluna tem NA o "isnan"
# apply aplica a função em todas as linhas

df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)

#competition_open_since_month

df1['competition_open_since_month']= df1.apply( lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

#competition_open_since_year 

df1['competition_open_since_year']= df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis = 1)
                               
#promo2_since_week
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'],axis=1)

#promo2_since_year       

df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

#promo_interval  
month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

df1['promo_interval'].fillna(0, inplace=True)

df1['month_map']= df1['date'].dt.month.map(month_map)


df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval']== 0 else 1 if x['month_map']in x['promo_interval'].split(',') else 0 , axis=1)                 


In [ ]:
df1.sample(5).T

## 1.6. Change

In [ ]:
df1.dtypes

In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)

df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

## 1.7. Descriptive Statistical

In [ ]:
#Separar as variáveis numéricas das categóricas

num_atributes = df1.select_dtypes(include = ['int64', 'float64'])
cat_atributes=df1.select_dtypes(include = ['object'])

## 1.7.1 Numerical Attributes

In [ ]:
#Central tendencian - mean, median

ct1 = pd.DataFrame( num_atributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_atributes.apply(np.median)).T

#Dispersion - std, min, max, range, skew, kurtosis

d1 = pd.DataFrame(num_atributes.apply(np.std)).T
d2 = pd.DataFrame(num_atributes.apply(min)).T
d3 = pd.DataFrame(num_atributes.apply(max)).T
d4 = pd.DataFrame(num_atributes.apply(lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame(num_atributes.apply(lambda x: x.skew() )).T
d6 = pd.DataFrame(num_atributes.apply(lambda x: x.kurtosis() )).T

#Concatenate
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes','min', 'max', 'range', 'mean', 'median', 'std', 'skew','kurtosis']
m

In [ ]:
#sns.displor('sales')

## 1.7.2 Categorical Attributes

In [ ]:
cat_atributes.apply(lambda x: x.unique().shape[0])

In [ ]:
# Faço com o boxplot